In [6]:
from common_nba_api_functions import rate_limited_request, get_season_start_and_end
from common_db_functions import get_players_filtered_by_team_playoff_participation, get_players_from_db
from manipulation import get_date_n_days_ago

season = "2023-24"
season_type = "Playoffs"
season_start, season_end = get_season_start_and_end(season)
# players = get_players_filtered_by_team_playoff_participation(season, True)
players = get_players_from_db()
players = [player for player in players if player.team_id == 1610612754]
print(players[0].name)
player_ids = [player.player_id for player in players]
print(f"num eligible players: {len(player_ids)}")
end = get_date_n_days_ago(1)
start = get_date_n_days_ago(35)




2023-24 season's first game: 2023-10-05
The 2023-24 season's last game: 2024-05-11
Tyrese Haliburton
num eligible players: 18


In [7]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from sqlalchemy.exc import SQLAlchemyError
from db_config import get_session
from models import GameStats, Player
from common_nba_api_functions import rate_limited_request
session = get_session()



def fetch_and_store_player_game_stats(player_id):
    try:
        player = session.query(Player).filter(Player.player_id == player_id).one_or_none()
        print(f"Fetching data for player: {player.name}")
        gamelog = playergamelog.PlayerGameLog(
            player_id=player_id, 
            season=season, 
            season_type_all_star=season_type, 
            date_from_nullable=start, 
            date_to_nullable=end
            )
        games_df = gamelog.get_data_frames()[0]
        print(games_df.head())
        games = []
        for index, game in games_df.iterrows():
            print(f"Adding new game stat for game ID: {game['Game_ID']}")
            new_game_stat = GameStats(
                player_id=player.id,
                game_id=game['Game_ID'],
                points=game['PTS'],
                assists=game['AST'],
                rebounds=game['REB'],
                steals=game['STL'],
                blocks=game['BLK'],
                turnovers=game['TOV'],
                fouls=game['PF'],
                minutes=game['MIN'],
                fg_made=game['FGM'],
                fg_attempts=game['FGA'],
                fg_percentage=game['FG_PCT'],
                fg3_made=game['FG3M'],
                fg3_attempts=game['FG3A'],
                fg3_percentage=game['FG3_PCT'],
                ft_made=game['FTM'],
                ft_attempts=game['FTA'],
                ft_percentage=game['FT_PCT'],
                plus_minus=game['PLUS_MINUS']
            )
            games.append(new_game_stat)
        session.add_all(games)
        session.commit()

    except SQLAlchemyError as e:
        print(f"Database error: {e}")
        session.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # It's a good practice to ensure the session is closed properly
        session.close()

for player_id in player_ids:
    rate_limited_request(1)
    fetch_and_store_player_game_stats(player_id)

    # Close session
    session.close()

Fetching data for player: Tyrese Haliburton
  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     42023    1630169  0042300213  MAY 10, 2024  IND vs. NYK  W   38   14   
1     42023    1630169  0042300212  MAY 08, 2024    IND @ NYK  L   35   11   
2     42023    1630169  0042300211  MAY 06, 2024    IND @ NYK  L   36    2   
3     42023    1630169  0042300126  MAY 02, 2024  IND vs. MIL  W   32    7   
4     42023    1630169  0042300125  APR 30, 2024    IND @ MIL  L   34    6   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   26   0.538  ...     4    4    7    2    1    2   1   35          16   
1   19   0.579  ...     4    6    9    3    0    2   0   34         -17   
2    6   0.333  ...     2    2    8    4    0    3   0    6         -12   
3   16   0.438  ...     5    6   10    0    0    2   1   17          12   
4   14   0.429  ...     3    4    6    0    0    2   1   16         -14   

   VIDEO_AVAILABLE  
0              